In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from pykrx import stock
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='Malgun Gothic')
sns.set()

#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)


1. 매매전략
    - MACD
        - signal과 MACD 곡선이 크로스 되는 순간이 매수/매도 신호
        - 기본값 : 12일(단기), 26일(장기), 9일(signal)
        - signal 곡선 = 9일 MACD 이동평균선
        - MACD 곡선 = (단기 이동평균- 장기이동평균)
        - Golden cross : MACD 곡선이 signal 곡선 상향 돌파 == 매수신호
        - Dead cross : MACD 곡선이 signal 곡선 하향 돌파 == 매도신호
    - RSI
        - RSI = ((n일 동안의 종가 상승 분 평균(AU))/(n일 동안의 종가 상승 분 평균 + n일 동안의 종가 하락분 평균(AD))) * 100
        - RSI 30%으로 떨어지면 초과매도 국면 == 매수
        - RSI 70%를 넘어서면 초과매수 국면 == 매도 -> 클수록 상승세


# Data Load

In [2]:
from pykrx import stock
from pykrx import bond

df =    stock.get_market_ohlcv('20200101', '20211231', '005930')[['시가','고가','저가','종가','거래량']]
# 삼성 =  stock.get_market_ohlcv('20211201','20221231', '005930')[['종가']]
df.columns=['open', 'high', 'low', 'close', 'volume']
df.head()

,open,high,low,close,volume
날짜,,,,,
2020-01-02,55500,56000,55000,55200,12993228
2020-01-03,56000,56600,54900,55500,15422255
2020-01-06,54900,55600,54600,55500,10278951
2020-01-07,55700,56400,55600,55800,10009778
2020-01-08,56200,57400,55900,56800,23501171


# RSI

- Golden cross : MACD 곡선이 signal 곡선 상향 돌파 == 매수신호
- Dead cross : MACD 곡선이 signal 곡선 하향 돌파 == 매도신호

In [3]:
import talib.abstract as ta

macd, macdsignal, macdhist= ta.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)

df['macd']=macd
df['macdsignal']=macdsignal
df['macdhist']=macdhist

In [4]:
Golden_cross=df[df['macd']>df['macdsignal']]
Dead_cross=df[df['macd']<df['macdsignal']]

- RSI = ((n일 동안의 종가 상승 분 평균(AU))/(n일 동안의 종가 상승 분 평균 + n일 동안의 종가 하락분 평균(AD))) * 100
- RSI 30%으로 떨어지면 초과매도 국면 == 매수
- RSI 70%를 넘어서면 초과매수 국면 == 매도 -> 클수록 상승세

In [5]:
df['rsi']=ta.RSI(df['close'],timeperiod=14)

In [6]:
buy=df[df['rsi']<30.0]
sell=df[df['rsi']>70.0]

In [7]:
df[df.index=='2021-03-22']

,open,high,low,close,volume,macd,macdsignal,macdhist,rsi
날짜,,,,,,,,,
2021-03-22,82000,82300,81700,82000,12670506,-238.336385,-231.555406,-6.780979,47.694227


# STOCHSTIC

In [8]:
import talib
from talib.abstract import *

slowk, slowd = talib.STOCH(df['high'], df['low'], df['close'], 
                           fastk_period=5, 
                           slowk_period=3, slowk_matype=0, 
                           slowd_period=3, slowd_matype=0)

In [9]:
fastk, fastd = talib.STOCHF(df['high'], df['low'], df['close'],  
                            fastk_period=5, 
                            fastd_period=3, fastd_matype=0)

In [10]:
df['fastk']=fastk
df['fastd']=fastd

In [11]:
df[(df['macdhist'] * df['macdhist'].shift() < 0) & (df['fastk']<=20.0)] # 매수신호
df[(df['macdhist'] * df['macdhist'].shift() > 0) & (df['fastk']>=70.0)] # 매도신호

,open,high,low,close,volume,macd,macdsignal,macdhist,rsi,fastk,fastd
날짜,,,,,,,,,,,
2020-03-04,54800,57600,54600,57400,24765728,-920.092098,-525.930768,-394.161330,49.015842,95.000000,59.919786
2020-03-05,57600,58000,56700,57800,21698990,-795.011690,-579.746953,-215.264737,50.521484,95.454545,81.131907
2020-03-24,43850,46950,43050,46950,49801908,-3560.309359,-2847.165222,-713.144138,39.283954,76.859504,38.542368
2020-03-25,48950,49600,47150,48650,52735922,-3252.239405,-2928.180058,-324.059347,43.614444,86.986301,55.522298
2020-03-26,49000,49300,47700,47800,42185129,-3041.617766,-2950.867600,-90.750166,42.001380,75.000000,79.615269
...,...,...,...,...,...,...,...,...,...,...,...
2021-12-22,78900,79400,78800,79400,17105892,1524.815981,1421.376948,103.439033,68.281463,100.000000,73.649539
2021-12-23,79800,80000,79300,79900,13577498,1614.784497,1460.058458,154.726039,69.794361,96.875000,92.897727
2021-12-24,80200,80800,80200,80500,12086380,1714.733795,1510.993525,203.740270,71.548134,92.500000,96.458333


In [12]:
len(df)

496

In [13]:
df['macdhist_shift']=df['macdhist'] * df['macdhist'].shift()

In [14]:
# np.where((df['macd'][0]>df['macdsignal'][0])&(df['rsi'][0]<20.0), True, False)

In [20]:
basic_asset=1000000
sell_price=0
buy_price=0
buy_signal=False
ret_list=[]
for idx, row in df.iterrows() :
    if basic_asset != 0:
        buy_signal = np.where((df['macdhist_shift'][idx] < 0) & (df['fastk'][idx]<=20.0), True, False)
        sell_signal = np.where((df['macdhist_shift'][idx] > 0) & (df['fastk'][idx]>=70.0), True, False)
        # print(buy_signal, sell_signal)
        if buy_signal:
            buy_price=df['close'].loc[idx]
            # print(buy_price)
        if sell_signal :  
            sell_price=df['close'].loc[idx]
            # print(sell_price)
        ret=sell_price-buy_price
        basic_asset+=ret
        ret_list.append(ret) 
    elif basic_asset <= 0:
        break
total_ret=np.sum(np.array(ret_list))
final_asset= basic_asset+ total_ret
final_ret=(final_asset/basic_asset)*100

In [21]:
final_asset,final_ret

(12111600, 184.74633149272398)

In [22]:
df.head()

,open,high,low,close,volume,macd,macdsignal,macdhist,rsi,fastk,fastd,macdhist_shift
날짜,,,,,,,,,,,,
2020-01-02,55500,56000,55000,55200,12993228,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,56000,56600,54900,55500,15422255,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06,54900,55600,54600,55500,10278951,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-07,55700,56400,55600,55800,10009778,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-08,56200,57400,55900,56800,23501171,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.dropna(inplace=True,axis=0)

In [ ]:

import numpy as np

class MyBackTesting :
    def __init__(self, daily_ohlcv, start_cash) :
        self.daily_ohlcv = daily_ohlcv      # 일봉 데이터
        # self.fee = 0.0011                   # 수수료 ( calculate_fee.xlsx 참고 )
        self.buy_signal = False             # 매수 신호
        self.sell_signal = False
        
        self.start_cash   = start_cash      # 시작 자산
        self.current_cash = start_cash      # 현재 자산
        self.highest_cash = start_cash      # 자산 최고점
        self.lowest_cash  = start_cash      # 자산 최저점

        self.ror = 1                        # 수익률
        self.accumulated_ror = 1            # 누적 수익률
        self.mdd = 0                        # 최대 낙폭

        self.trade_count = 0
        # self.buy_trade_count = 0
        # self.sell_trade_count = 0         # 거래횟수
        self.win_count = 0                  # 승리횟수

    def my_execute(self) : # 전략
        K = 0.5 # 수익률
        
        # 변동폭 ( 고가 - 저가 )
        self.daily_ohlcv['range'] = self.daily_ohlcv['high'] - self.daily_ohlcv['low']
        # 목표매수가 ( 시가 + 변동폭 * K )
        self.daily_ohlcv['targetPrice'] = self.daily_ohlcv['open'] + self.daily_ohlcv['range'].shift(1) * K

        for idx, row in self.daily_ohlcv.iterrows() :

            self.buy_signal = np.where((df['macdhist_shift'][idx] < 0) & (df['fastk'][idx]<=20.0), True, False)
            self.sell_signal = np.where((df['macdhist_shift'][idx] > 0) & (df['fastk'][idx]>=70.0), True, False)
            
            if self.buy_signal:
            #     self.buy_trade_count += 1
            #     self.buy_signal=False
            # elif self.sell_signal:
            #     self.sell_trade_count += 1

            # 거래 횟수
                self.trade_count+=1
                # 수익률 계산
                self.ror = row['close'] / row['targetPrice'] 
                # 촣거래 횟수
                # self.total_trade_count=self.buy_trade_count+self.sell_trade_count

                # 누적 수익률 계산
                self.accumulated_ror = (1+self.ror).cumprod()-1

                # 현재 자산 갱신
                self.current_cash *= self.ror
                # 자산 최고점 갱신
                self.highest_cash = max(self.highest_cash, self.current_cash)
                # 자산 최저점 갱신
                self.lowest_cash = min(self.lowest_cash, self.current_cash)
                # 최대 낙폭 계산
                dd = (self.highest_cash - self.current_cash) / self.highest_cash * 100
                self.mdd = max(self.mdd, dd)
        print('='*40)
        print('테스트 결과')
        print('-'*40)
        print('총 거래 횟수 : %s' %self.trade_count)
        print('누적 수익률 : %s' %self.accumulated_ror)
        print('현재 잔액 : %s' % self.current_cash)
        print('최고 잔액 : %s' % self.highest_cash)
        print('최저 잔액 : %s' % self.lowest_cash)
        print('최대 낙폭 (MDD) : %s' % self.mdd)
        print('='*40)


backtest = MyBackTesting(df, 1000000)
backtest.my_execute()